In [9]:
from langchain_chroma import Chroma

from pyprojroot import here
from langchain_ollama import OllamaEmbeddings
from ollama import Client
from dotenv import load_dotenv
from pprint import pprint
load_dotenv()

True

#### Config

In [11]:
EMBEDDING_MODEL = "nomic-embed-text"
VECTORDB_DIR = "data/airline_policy_vectordb"
K=2

#### Load the VectorDB

In [12]:
vectordb = Chroma(
    collection_name="rag-chroma",
    persist_directory=str(here(VECTORDB_DIR)),
    embedding_function=OllamaEmbeddings(model=EMBEDDING_MODEL)
)
print("Number of vectors in vectordb:",
      vectordb._collection.count(), "\n\n")

Number of vectors in vectordb: 22 




#### Sample Query

In [13]:
message = "What is the cancelation rule for a flight ticket at swiss airline policy?"

#### Perform the vector Search

In [14]:
docs = vectordb.similarity_search(message, k=K)

In [15]:
docs

[Document(metadata={'page': 9, 'source': '/home/kacper/Documents/Projects/rag_sql_agent/data/unstructured_docs/swiss_airline_policy/swiss_faq.pdf'}, page_content="for a refund or may only be able to receive a partial refund. If you booked your flight through a third-party website or\ntravel agent, you may need to contact them directly to cancel your flight. Always check the terms and conditions of your\nticket to make sure you understand the cancellation policy and any associated fees or penalties. If you're cancelling your\nflight due to unforeseen circumstances such as a medical emergency or a natural disaster , Swiss Air may of fer you\nspecial exemptions or accommodations. What is Swiss Airlines 24 Hour Cancellation Policy? Swiss Airlines has a 24"),
 Document(metadata={'page': 8, 'source': '/home/kacper/Documents/Projects/rag_sql_agent/data/unstructured_docs/swiss_airline_policy/swiss_faq.pdf'}, page_content="How to Cancel a Swiss Air Flight: 877-\n5O7-7341 Step-by-Step Guide\nSwi

#### Prepare the prompt for the GPT model

In [16]:
question = "# User new question:\n" + message
retrieved_content = ""
for doc in docs:
    retrieved_content += f"{doc.page_content}\n\n"
prompt = f"# Content:\n{retrieved_content}\n\n{question}"

In [17]:
pprint(prompt)

('# Content:\n'
 'for a refund or may only be able to receive a partial refund. If you booked '
 'your flight through a third-party website or\n'
 'travel agent, you may need to contact them directly to cancel your flight. '
 'Always check the terms and conditions of your\n'
 'ticket to make sure you understand the cancellation policy and any '
 "associated fees or penalties. If you're cancelling your\n"
 'flight due to unforeseen circumstances such as a medical emergency or a '
 'natural disaster , Swiss Air may of fer you\n'
 'special exemptions or accommodations. What is Swiss Airlines 24 Hour '
 'Cancellation Policy? Swiss Airlines has a 24\n'
 '\n'
 'How to Cancel a Swiss Air Flight: 877-\n'
 '5O7-7341 Step-by-Step Guide\n'
 'Swiss Air is a premium airline based in Switzerland that of fers a range of '
 'domestic and international flights to\n'
 'passengers. However , sometimes situations arise where passengers may need '
 'to cancel their flights. In such cases, it is\n'
 'import

In [ ]:
client = Client()
response = client.chat(
    model='llama3-groq-tool-use',
    messages=[
        {
            "role": "system",
            "content": "You will receive a user's query and possible content where the answer might be. If the answer is found, provide it, if not, state that the answer does not exist."
        },
        {
            "role": "user",
            "content": prompt
        }
    ]
)

assistant_response = response['message']['content']

In [20]:
pprint(assistant_response)

("According to Swiss Air's policy, if you have booked a flexible fare type "
 'like Flex or Business Flex, you can cancel your flight up to 24 hours before '
 'departure without any penalty. For other fare types, cancellation fees will '
 'apply depending on the time of cancellation. The closer you cancel to the '
 'departure date, the higher the cancellation fee.')


#### Rag Tool designe using LangChain

In [21]:
from langchain_core.tools import tool

@tool
def lookup_swiss_airline_policy(query: str)->str:
    """Search within the Swiss Airline's company policies to check whether certain options are permitted. Input should be a search query."""
    vectordb = Chroma(
    collection_name="rag-chroma",
    persist_directory=str(here(VECTORDB_DIR)),
    embedding_function=OllamaEmbeddings(model=EMBEDDING_MODEL)
    )
    docs = vectordb.similarity_search(query, k=K)
    return "\n\n".join([doc.page_content for doc in docs])

In [22]:
print(lookup_swiss_airline_policy.name)
print(lookup_swiss_airline_policy.args)
print(lookup_swiss_airline_policy.description)

lookup_swiss_airline_policy
{'query': {'title': 'Query', 'type': 'string'}}
Search within the Swiss Airline's company policies to check whether certain options are permitted. Input should be a search query.


In [23]:
pprint(lookup_swiss_airline_policy.invoke("can I cancel my ticket?"))

('for a refund or may only be able to receive a partial refund. If you booked '
 'your flight through a third-party website or\n'
 'travel agent, you may need to contact them directly to cancel your flight. '
 'Always check the terms and conditions of your\n'
 'ticket to make sure you understand the cancellation policy and any '
 "associated fees or penalties. If you're cancelling your\n"
 'flight due to unforeseen circumstances such as a medical emergency or a '
 'natural disaster , Swiss Air may of fer you\n'
 'special exemptions or accommodations. What is Swiss Airlines 24 Hour '
 'Cancellation Policy? Swiss Airlines has a 24\n'
 '\n'
 'circumstances such as bad weather or political unrest, Swiss Airlines may '
 'not be obligated to of fer any compensation. In\n'
 "summary , Swiss Airlines' cancellation policy varies depending on your fare "
 'type and the time of cancellation. T o avoid any\n'
 "unnecessary fees or charges, it's important to familiarise yourself with the "
 'terms 